In [ ]:
import os
from os import path
import numpy as np
import nibabel as nib
from tqdm import tqdm
from torch.utils.data import Dataset
from glob import glob
import random
import torch
import torchio as tio
import matplotlib.pyplot as plt
from dataset_tio import Normalize, ImagePair, calculate_overlap
import torchvision
from models.generator import GeneratorRRDB
from trainer_org import LitTrainer as LitTrainer_org
from trainer_gan import LitTrainer as LitTrainer_gan
import pytorch_lightning as pl
from models.discriminator import Discriminator
from models.feature_extractor import FeatureExtractor
from torchvision.utils import save_image
from torchsummary import summary
import time
from utils import save_subject

print(os.getcwd())

In [ ]:
# config = {
#     'learning_rate': 0.0001,
#     'patch_size': 224,
#     'batch_size': 16,
#     'patients_frac': 0.5,
#     'patch_overlap': 0.5,
#     'optimizer': 'adam',
#     'edge_loss': 2,
# }
# ckpt_filename = 'checkpoint_{}_{}_{}_{}_{}_{}'.format(config['patch_size'],
#                                                       config['batch_size'],
#                                                       config['patients_frac'],
#                                                       config['edge_loss'],
#                                                       config['optimizer'],
#                                                       config['learning_rate'],
#                                                       )

config = model.hparams.config
# img_name = 'gen_{}_{}_{}_{}_{}_{}_{}'.format(config['patch_size'],
#                                           config['batch_size'],
#                                           config['patients_frac'],
#                                           config['edge_loss'],
#                                           config['optimizer'],
#                                           # config['learning_rate'],
#                                           config['content'],
#                                           config['adversarial'],
#                                           )

# img_name = 'real_gen_{}'.format(config['patch_size'])
# subject['gen_hr'].save(os.path.join('output', img_name+'_2.dcm'))
# subject['gen_hr'].save(os.path.join('output'))#, img_name+'_2.dcm'))

In [ ]:
print(pl.__version__)

In [ ]:
import json
dict = {'alpha_content': 0.01,
        'patients_frac': 0.1,
        }

fname = json.dumps(dict)

# fname = "{'alpha_content': 0.01}"
# fname = "{'alpha_content': 0.01, 'patients_frac': 0.1}"

chars = ['"', '{', '}', '. ', '_']

for char in chars:
	fname = fname.replace(char, '')
fname = fname.replace(': ', '_')
fname = fname.replace(', ', '-')
fname = fname.replace('.', ',')



print(fname)

In [ ]:
import json
dict = {'alpha_content': 0.01,
        'patients_frac': 0.1,
        }

print(json.dumps(dict))


In [ ]:
from tensorboard.backend.event_processing import event_accumulator
import cv2

def save_images_from_event(path):
    event_acc = event_accumulator.EventAccumulator(path, size_guidance={'images': 0})
    event_acc.Reload()
    path, fname = os.path.split(path)
    for tag in event_acc.Tags()['images']:
        events = event_acc.Images(tag)
        tag_name = tag.replace('/', ' ')
        tag_path = os.path.join(path, 'images', tag_name)
        os.makedirs(tag_path, exist_ok=True)
        for index, event in enumerate(events):
            s = np.frombuffer(event.encoded_image_string, dtype=np.uint8)
            image = cv2.imdecode(s, cv2.IMREAD_GRAYSCALE)
            fname = '{:04}.jpg'.format(index)
            cv2.imwrite(os.path.join(tag_path, fname), image)

In [ ]:
path = '/mnt/beta/djboonstoppel/Code/log/wgan/version_6/events.out.tfevents.1644858303.shire.2157378.0'
save_images_from_event(path)


In [ ]:
config = {
    'learning_rate': 1e-4,
    # 'patch_size': args.patch_size,
    # 'batch_size': args.batch_size,
    'patients_frac': 0.5,
    'patch_overlap': 0.5,
    'optimizer': 'adam',
    'edge_loss': 2,
    'b1': 0.9,
    'b2': 0.5,
    'alpha_content': 1,
}

# print ("{:<8} {:<15} {:<10}".format('Name','Age','Percent'))

for key in config:
    # name, age, perc = v
    print ("{:<13}: {:<10} ".format(key, config[key]))

In [ ]:
with torch.no_grad():
    i = 0
    for patches_batch in patch_loader:
        i += 1
        imgs_lr, imgs_hr = patches_batch['LR'][tio.DATA].squeeze(4), patches_batch['HR'][tio.DATA].squeeze(4)
        if i == 5:
            imgs_lr = imgs_lr[200,0,:,:]*std
            save_image(imgs_lr, 'imgs_lr.png')
            print(imgs_lr.shape)
            np.savetxt('imgs_lr.csv', imgs_lr.numpy(), delimiter=',')
    print(i)
        # gen_hr = model(imgs_lr.to(device)).unsqueeze(4)
        # locations = patches_batch[tio.LOCATION]
        # aggregator.add_batch(gen_hr, locations)

In [ ]:
config = {
    'ragan': False,
    'learning_rate': 1e-4,
    'patch_size': 64,
    'batch_size': 256,
    'patients_frac': 0.5,
    'patch_overlap': 0.5,
    'optimizer': 'adam',
    'edge_loss': 2,
    'b1': 0.9,
    'b2': 0.5,
    'alpha_content': 1,
}
if config['ragan']:
    print('RaGAN')

In [ ]:
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        self.input_shape = input_shape
        in_channels, in_height, in_width = self.input_shape
        patch_h, patch_w = int(in_height / 2 ** 4), int(in_width / 2 ** 4)
        self.output_shape = (1, patch_h, patch_w)

        def discriminator_block(in_filters, out_filters, first_block=False):
            layers = []
            layers.append(nn.Conv2d(in_filters, out_filters, kernel_size=3, stride=1, padding=1))
            if not first_block:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            layers.append(nn.Conv2d(out_filters, out_filters, kernel_size=3, stride=2, padding=1))
            layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        layers = []
        in_filters = in_channels
        for i, out_filters in enumerate([64, 128, 256, 512]):
            layers.extend(discriminator_block(in_filters, out_filters, first_block=(i == 0)))
            in_filters = out_filters

        layers.append(nn.Conv2d(out_filters, 1, kernel_size=3, stride=1, padding=1))

        self.model = nn.Sequential(*layers)

    def forward(self, img):
        return self.model(img)

In [ ]:
discriminator = Discriminator(input_shape=(3, 256, 256))

summary(discriminator, input_size=(3, 256, 256), batch_size=32, device='cpu')

In [ ]:
import torch.nn as nn
class DenseResidualBlock(nn.Module):
    """
    The core module of paper: (Residual Dense Network for Image Super-Resolution, CVPR 18)
    """

    def __init__(self, filters, res_scale=0.2):
        super(DenseResidualBlock, self).__init__()
        self.res_scale = res_scale

        def block(in_features, non_linearity=True):
            layers = [nn.Conv2d(in_features, filters, 3, 1, 1, bias=True)]
            if non_linearity:
                layers += [nn.LeakyReLU()]
            return nn.Sequential(*layers)

        self.b1 = block(in_features=1 * filters)
        self.b2 = block(in_features=2 * filters)
        self.b3 = block(in_features=3 * filters)
        self.b4 = block(in_features=4 * filters)
        self.b5 = block(in_features=5 * filters, non_linearity=False)
        self.blocks = [self.b1, self.b2, self.b3, self.b4, self.b5]

    def forward(self, x):
        inputs = x
        for block in self.blocks:
            out = block(inputs)
            inputs = torch.cat([inputs, out], 1)
        return out.mul(self.res_scale) + x


class ResidualInResidualDenseBlock(nn.Module):
    def __init__(self, filters, res_scale=0.2):
        super(ResidualInResidualDenseBlock, self).__init__()
        self.res_scale = res_scale
        self.dense_blocks = nn.Sequential(
            DenseResidualBlock(filters), DenseResidualBlock(filters), DenseResidualBlock(filters)
        )

    def forward(self, x):
        return self.dense_blocks(x).mul(self.res_scale) + x
class GeneratorRRDB(nn.Module):
    def __init__(self, channels, filters=64, num_res_blocks=16, num_upsample=2):
        super(GeneratorRRDB, self).__init__()

        # First layer
        self.conv1 = nn.Conv2d(channels, filters, kernel_size=3, stride=1, padding=1)
        # Residual blocks
        self.res_blocks = nn.Sequential(*[ResidualInResidualDenseBlock(filters) for _ in range(num_res_blocks)])
        # Second conv layer post residual blocks
        self.conv2 = nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=1)
        # Upsampling layers
        upsample_layers = []
        for _ in range(num_upsample):
            upsample_layers += [
                nn.Conv2d(filters, filters * 4, kernel_size=3, stride=1, padding=1),
                nn.LeakyReLU(),
                nn.PixelShuffle(upscale_factor=2),
            ]
        self.upsampling = nn.Sequential(*upsample_layers)
        # Final output block
        self.conv3 = nn.Sequential(
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(filters, channels, kernel_size=3, stride=1, padding=1),
        )

    def forward(self, x):
        out1 = self.conv1(x)
        out = self.res_blocks(out1)
        out2 = self.conv2(out)
        out = torch.add(out1, out2)
        out = self.upsampling(out)
        out = self.conv3(out)
        return out

In [ ]:
generator = GeneratorRRDB(1, filters=64, num_res_blocks=1, num_upsample=0)

summary(generator, input_size=(1, 64, 64), batch_size=32, device='cpu')

